# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.
  
![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

In [1]:
%pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# working directory 아래 `.env` 파일을 생성
# `.gitignore` 파일에 `.env` 을 등록해서 github에 올라가지 않도록 한다.
# 이름=값 형식으로 환경변수를 설정.
##  HUGGINGFACE_API_KEY="받은 Access Token"

In [3]:
from dotenv import load_dotenv
load_dotenv()
# working director에서 환경변수 파일(default: `.env`)찾아서 
# 그 파일에 설정된 값을 O/S의 환경변수로 등록해준다.

True

In [1]:
# 환경변수 값을 읽기
import os
os.getenv("JAVA_HOME")

'C:\\Documents\\jdk-24.0.1'

In [ ]:
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
# os.environ['HUGGINGFACE_API_KEY']
# print(hf_api_key)


# Naver 영화댓글 분류

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [9]:
# NSMC 데이터셋 로딩
import datasets
from datasets import load_dataset

nsmc = load_dataset("e9t/nsmc", trust_remote_code=True)
type(nsmc)

datasets.dataset_dict.DatasetDict

In [ ]:
# Dataset: 데이터셋
# DatasetDict: Dataset들을 모아놓은 dict기반의 자료구조. 
#              train/valid/test set들을 모아서 제공할 때 사용용

In [10]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [11]:
nsmc.keys()

dict_keys(['train', 'test'])

In [12]:
trainset = nsmc['train']
testset = nsmc['test']

In [ ]:
# dataset이 표데이터일 경우 features는 컬럼명.
trainset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [17]:
# feature 조회
trainset['document'][:5]
trainset['label'][:5]
trainset['id'][:5]

['9976970', '3819312', '10265843', '9045019', '6483659']

In [18]:
# datasets.Dataset 을 다른 형식으로 변환
# dataset객체.to_xxxxxx()
df = trainset.to_pandas()
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [20]:
# 다른 형식으로 저장된 data들을 datasets.Dataset으로 변환
# datasets.Dataset.from_xxxxx(데이터셋)
d = datasets.Dataset.from_pandas(df.head(100))
d

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 100
})

In [21]:
# Sampling - train: 10_000, test: 5_000

#dataset.shuffle(): 섞어준다. select([조회할 index들])
sample_train = trainset.shuffle().select(range(10_000))
sample_test = testset.shuffle().select(range(5_000))

In [22]:
print(sample_train)
print(sample_test)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10000
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
})


## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "beomi/kcbert-base"  # Feature Extractor
# 토크나이저, 분류 모델을 로딩. - 토크나이저와 모델을 같은 id것을 받아야함.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
# num_labels: 분류할 클래스의 개수 (긍정, 부정)
# model_id: kcbert-base - pretrained 된 Feature Extractor
# model: Estimator는 학습안된 layer로 구성.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

In [31]:
train_X = sample_train['document']  # sample_train -> trainset(전체)
train_y = sample_train['label']

test_X = sample_test['document']    # sample_test -> testset (전체)
test_y = sample_test['label']

In [33]:
# X(댓글)을 토큰화
train_encoding = tokenizer(train_X, return_tensors="pt", padding=True) # train_X에서 가장 긴 문장을 기준으로 padding처리.
test_encoding = tokenizer(test_X, return_tensors="pt", padding=True)

In [ ]:
train_encoding.keys()
train_encoding['input_ids'].shape # [10000:batch-문서수, 121:토큰수수]

torch.Size([10000, 121])

In [48]:
d = {
    "input_ids":[1, 2, 3, 4],
    "token_type_ids":[10, 20, 30, 40],
    "attention_mask": [100, 200, 300, 400]
}
idx = 3
{k:v[idx] for k, v in d.items()}
# data = {"input_ids":[4], "token_type_ids":[40], "attention_mask":[400]}

{'input_ids': 4, 'token_type_ids': 40, 'attention_mask': 400}

In [49]:
################################
# Pytorch의 Dataset을 정의, 생성
################################
import torch
from torch.utils.data import Dataset

class NSMCDataset(Dataset):

    def __init__(self, comments, labels):
        """
        Args:
            comments(dict) : tokenizer로 토큰화한 input data들 
            labels(list) : 정답 라벨들.
        """
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.labels) 

    def __getitem__(self, index):
        """
        index번째 데이터를 반환
        BERT 모델 입력 형식에 맞춰서 반환. input_ids, token_type_ids, attention_mask + label
        Args: 
            index(int)
        Returns:
            dictionary - input_ids, token_type_ids, attention_mask, label  
                         입력데이터와 정답 label을 딕셔너리에 묶어서 반환.
        """
        data = {key:value[index] for key, value in self.comments.items()}
        data['labels'] = torch.tensor([self.labels[index]], dtype=torch.int64)

        return data


In [50]:
train_set = NSMCDataset(train_encoding, train_y)
test_set = NSMCDataset(test_encoding, test_y)

In [51]:
len(train_set), len(test_set)

(10000, 5000)

In [52]:
train_set[0]

{'input_ids': tensor([    2, 18104, 20301, 16655, 12728,  4180, 17079,    17, 10631,  4019,
         11219,  8069,   360,     5,  8098,  2332, 13798,  4337,  4324, 13326,
            32,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0]),
 'token_type_ids': te

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments는 Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [ ]:
from transformers import TrainingArguments, Trainer

N_EPOCHS = 1 
BATCH_SIZE = 64

# 어떻게 학습할지에 대한 설정.
train_args = TrainingArguments(
    output_dir="models/nsmc",   # train 모델을 저장할 디렉토리 경로.
    num_train_epochs=N_EPOCHS,  # 학습 에폭수
    per_device_train_batch_size=BATCH_SIZE, # 학습할때 batch size
    per_device_eval_batch_size=BATCH_SIZE,  # 검증시 batch size

    eval_strategy="epoch",  # 평가 전략 - "no", "step", "epoch"
    save_strategy="epoch", # 저장 전략.- "no", "step", "epoch"

    save_total_limit=1,  # 저장할 모델의 최대 개수.
    load_best_model_at_end=True, # 학습 종료 후 검증결과가 가장 좋은 모델을 저장 + load. 
                                 # True: eval_strategy, save_strategy가 같아야한다.
    metric_for_best_model="eval_loss", # best model을 결정할 검증 기준 평가지표.
    greater_is_better=False, # 검증 평가지표값이 높아야 좋은지 낮아야 좋은지.

    report_to="none",

    # huggingface-hub에 올리는 설정
    # push_to_hub=True, 
    # hub_model_id="kcbert-nsmc-10000", # 모델 ID
    # hub_token="Access Token"
)

In [ ]:
# 평가 함수 정의 - evaluate 패키지를 이용
#  huggingface에서 제공하는 라이브러리. 다양한 평가함수들을 제공.
%pip install evaluate scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [59]:
import evaluate

# 정확도 평가함수.
acc_fn = evaluate.load("accuracy") # f1, recall, precision
# acc_fn

In [61]:
pred = torch.tensor([0, 1, 0, 1])
ref = torch.tensor([0, 1, 0, 0])
acc_fn.compute(predictions=pred, references=ref)

{'accuracy': 0.75}

In [ ]:
import evaluate
def compute_metrics(pred):
    """
    모델이 학습하는 도중 예측값을 받아서 평가 점수를 계산하는 함수.
    Trainer에 의해서 호출될 함수.
    Args:
        pred(EvalPrediction) - 모델의 예측값,정답들을 묶어서 제공.
    Returs
        dictionary - key: 평가지표이름, value: 평가점수.
    """
    labels = pred.label_ids # 정답
    preds = pred.predictions.argmax(axis=-1)  # 모델의 출력값.
    metrics1 = evaluate.load("accuracy")
    metrics2 = evaluate.load("f1")

    acc = metrics1.compute(references=labels, predictions=preds)
    f1 = metrics2.compute(references=labels, predictions=preds)
    return {"accuracy":acc, "f1 score":f1}

In [ ]:
# Trainer 객체
trainer = Trainer(
    model=model,     # 학습 대상 모델
    args=train_args, # TrainingArguments
    train_dataset=train_set,  # 학습 데이셋. pytorch의 Dataset객체 - trainer.train()
    eval_dataset=test_set,    # 검증 데이터셋.   trainer.evaluate()
    compute_metrics=compute_metrics # loss 이외에 검증/평가시 확인할 지표를 계산하는 함수.
)

In [64]:
#  학습
trainer.train()

c:\Users\Playdata\miniconda3\envs\dl\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


TypeError: argmax() got an unexpected keyword argument 'dim'

In [65]:
trainer.evaluate()

c:\Users\Playdata\miniconda3\envs\dl\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TypeError: argmax() got an unexpected keyword argument 'dim'

In [66]:
# 저장 - local directory
## tokenizer와 model을 같이 같은 경로에 저장
save_path = "models/nsmc"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

## 모델 로드

In [67]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

save_path = "models/nsmc"
load_tokenizer = AutoTokenizer.from_pretrained(save_path)
load_model = AutoModelForSequenceClassification.from_pretrained(save_path)

# 추론

In [ ]:
sentence = ["이걸 영화라고 만든 거냐?", 
            "아무 기대 없이 봤는데 재미있네.", 
            "내가 감독이어도 이것보다 재미있게 만들겠다.", 
            "시간이 어떻게 가는 줄 모르고 봤다."]

In [69]:
from transformers import pipeline
pipe = pipeline(task="text-classification", tokenizer=load_tokenizer, model=load_model)
result = pipe(sentence)

Device set to use cpu


In [70]:
result

[{'label': 'LABEL_0', 'score': 0.9830850958824158},
 {'label': 'LABEL_1', 'score': 0.9721060991287231},
 {'label': 'LABEL_1', 'score': 0.8297526240348816},
 {'label': 'LABEL_0', 'score': 0.8027584552764893}]

In [ ]:
####### 학습한 모델, 토크나이저를 huggingface hub에 업로드#########
# 1. TrainingArguments에 설정해서 학습 도중/끝나면 자동으로 업로드 되게 설정.
# 2. model/tokenzer.push_to_hub("계정/모델id") 를 이용해서 수동으로 올린다.

# 로그인
from huggingface_hub import login
from dotenv import load_dotenv
# login() # Token에 Access Token 입력 후 Login 클릭
# login("Huggingface Access Token")
load_dotenv()  # .env 에 환경변수들을 로드.

True

In [76]:
import os
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
login(hf_api_key)

In [77]:
model_id = "kcbert-nsmc-10000"
load_tokenizer.push_to_hub(model_id)
load_model.push_to_hub(model_id)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\Playdata\miniconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--kgmyh--kcbert-nsmc-10000. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kgmyh/kcbert-nsmc-10000/commit/3a6c055491b795f1da5ddcb8223fbe9408c4e922', commit_message='Upload BertForSequenceClassification', commit_description='', oid='3a6c055491b795f1da5ddcb8223fbe9408c4e922', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kgmyh/kcbert-nsmc-10000', endpoint='https://huggingface.co', repo_type='model', repo_id='kgmyh/kcbert-nsmc-10000'), pr_revision=None, pr_num=None)

In [81]:
## huggingface-hub에 push한 모델 사용.
from transformers import pipeline
model_id = "kgmyh/kcbert-nsmc-10000"
pipe = pipeline(task='text-classification', model=model_id)

Device set to use cpu


In [82]:
pipe(["영화 너무 재미있어요.", "시간이 아깝다."])

[{'label': 'LABEL_1', 'score': 0.9968327879905701},
 {'label': 'LABEL_0', 'score': 0.9996107220649719}]